hello world


In [18]:
from sklearn.cluster import KMeans
from scipy.linalg import norm
import numpy as np
import scipy.io as scio
import math


data_test = scio.loadmat('data_test.mat')['data_test']
data_train = scio.loadmat( 'data_train.mat')['data_train']
label_train = scio.loadmat(  'label_train.mat')['label_train']

center_num = 10
model = KMeans(n_clusters=center_num, max_iter=1500)
model.fit(data_train)
centers = model.cluster_centers_


class RBF(object):
    global center_num

    def __init__(self, centers, iter_num, input_data, label, test_data):
        self.input_data = input_data
        self.label = label
        self.test_data = test_data
        self.iter_num = iter_num
        self.centers = centers
        self.w = np.array([np.random.rand(center_num+1)])

    def basis(self, data, center):
        sigma = np.amax(norm(data-center))/math.sqrt(2*center_num)
        return math.exp(-norm(data-center)**2/(2*sigma*sigma))

    def calcAct(self, input_data):
        act_val = np.zeros((input_data.shape[0], center_num))
        for xi, x in enumerate(input_data):
            for ci, c in enumerate(self.centers):
                act_val[xi][ci] = self.basis(c, x)
        return act_val

    def train(self):
        act_val = self.calcAct(self.input_data)
        w0_column = np.array([1 for _ in range(len(act_val))])
        act_val = np.insert(act_val, len(act_val[0]), values=w0_column, axis=1)
        output = act_val.dot(self.w.T)
        i = 0
        while i < self.iter_num:
            self.w = np.linalg.inv(act_val.T.dot(act_val)).dot(act_val.T).dot(self.label)
            output = act_val.dot(self.w)
            i += 1
        return output, act_val

    def test(self):
        act_val_test = self.calcAct(self.test_data)
        w0_column = np.array([1 for _ in range(len(act_val_test))])
        act_val_test = np.insert(act_val_test, 10, values=w0_column, axis=1)
        output = act_val_test.dot(self.w)
        return output

network = RBF(centers, 1000, data_train, label_train, data_test)
output, actval = network.train()
res = network.test()

In [5]:
from sklearn.cluster import KMeans
from scipy.linalg import norm
import numpy as np
import scipy.io as scio
import math

data_test = scio.loadmat( 'data_test.mat')['data_test']
data_train = scio.loadmat( 'data_train.mat')['data_train']
label_train = scio.loadmat( 'label_train.mat')['label_train']

center_num = 10
model = KMeans(n_clusters=center_num, max_iter=1500)
model.fit(data_train)
centers = model.cluster_centers_


class RBF(object):
    global center_num

    def __init__(self, centers, iter_num, input_data, label, test_data):
        self.input_data = input_data
        self.label = label
        self.test_data = test_data
        self.iter_num = iter_num
        self.centers = centers
        self.w = np.array([np.random.rand(center_num+1)])

    def basis(self, data, center):
        sigma = np.amax(norm(data-center))/math.sqrt(2*center_num)
        return math.exp(-norm(data-center)**2/(2*sigma*sigma))

    def calcAct(self, input_data):
        act_val = np.zeros((input_data.shape[0], center_num))
        for xi, x in enumerate(input_data):
            for ci, c in enumerate(self.centers):
                act_val[xi][ci] = self.basis(c, x)
        return act_val

    def train(self):
        act_val = self.calcAct(self.input_data)
        w0_column = np.array([1 for _ in range(len(act_val))])
        act_val = np.insert(act_val, len(act_val[0]), values=w0_column, axis=1)
        output = act_val.dot(self.w.T)
        i = 0
        while i < self.iter_num:
            self.w = np.linalg.inv(act_val.T.dot(act_val)).dot(act_val.T).dot(self.label)
            output = act_val.dot(self.w)
            i += 1
        return output, act_val

    def test(self):
        act_val_test = self.calcAct(self.test_data)
        w0_column = np.array([1 for _ in range(len(act_val_test))])
        act_val_test = np.insert(act_val_test, 10, values=w0_column, axis=1)
        output = act_val_test.dot(self.w)
        return output

network = RBF(centers, 1000, data_train, label_train, data_test)
output, actval = network.train()
res = network.test()

In [39]:
from sklearn.cluster import KMeans
from scipy.linalg import norm
import numpy as np
import scipy.io as scio
import math
from sklearn.model_selection import KFold


data_test = scio.loadmat( 'data_test.mat')['data_test']
data_train = scio.loadmat( 'data_train.mat')['data_train']
label_train = scio.loadmat( 'label_train.mat')['label_train']

center_num = 10
model = KMeans(n_clusters=center_num, max_iter=1000)
model.fit(data_train)
centers = model.cluster_centers_


class RBF(object):
    global center_num

    def __init__(self, centers, input_data, label, test_data):
        self.input_data = input_data
        self.label = label
        self.test_data = test_data
        self.centers = centers
        self.w = np.array([np.random.rand(center_num + 1)])
        self.sigma = self.getSigma(self.centers)
        self.threshold = 0

    def getSigma(self, centers):
        sigma_ = 0
        for i in centers:
            for j in centers:
                if abs(norm(i) - norm(j)) > sigma_:
                    sigma_ = abs(norm(i) - norm(j))
        return sigma_

    def basis(self, data, center):
        return math.exp(-norm(data - center) ** 2 / (2 * self.sigma ** 2))

    def calcAct(self, input_data):
        act_val = np.zeros((input_data.shape[0], center_num))
        for xi, x in enumerate(input_data):
            for ci, c in enumerate(self.centers):
                act_val[xi][ci] = self.basis(x, c)
        return act_val

    def getThreshold(self, output, label):
        acc = 0
        threshold = 0
        th_list = np.arange(-0.5, 0.55, 0.1)
        for i in th_list:
            acc_ = self.acc(output, i, label)
            print('threshold=%.2f' % i + ' | acc = %.5f' % acc_)
            if acc_ > acc:
                acc = acc_
                threshold = i
        print('*' * 20 + '\n')
        print('Choose %0.2f as threshold' % threshold + '\n')
        print('*' * 20 + '\n')
        self.threshold = threshold

    def acc(self, output, threshold, label, training=True):
        res = np.ones(output.shape)
        res[output > threshold] = 1
        res[output < threshold] = -1
        if training:
            return sum(res == label) / output.shape[0]
        else:
            return res

    def train(self):
        act_val = self.calcAct(self.input_data)
        w0_column = np.array([1 for _ in range(len(act_val))])
        act_val = np.insert(act_val, len(act_val[0]), values=w0_column, axis=1)
        self.w = np.linalg.inv(act_val.T.dot(act_val)).dot(act_val.T).dot(self.label)
        output = act_val.dot(self.w) 
        self.getThreshold(output, label=self.label)
        return output

    def valid(self, data, label):
        act_val_test = self.calcAct(data)
        w0_column = np.array([1 for _ in range(len(act_val_test))])
        act_val_test = np.insert(act_val_test, len(act_val_test[0]), values=w0_column, axis=1)
        output = act_val_test.dot(self.w)
        self.getThreshold(output,label)
        return output, self.acc(output, self.threshold, label)

    def test(self):
        act_val_test = self.calcAct(self.test_data)
        w0_column = np.array([1 for _ in range(len(act_val_test))])
        act_val_test = np.insert(act_val_test, len(act_val_test[0]), values=w0_column, axis=1)
        output = act_val_test.dot(self.w)
        print('Label of testing data are:')
        print(np.squeeze(self.acc(output, self.threshold, None, False)))
        return output, self.acc(output, self.threshold, None, False)


K = 4
kf = KFold(n_splits=K, shuffle=False)
train_index, valid_index = next(kf.split(label_train))

data_valid = data_train[valid_index]
label_valid = label_train[valid_index]
data_train = data_train[train_index]
label_train = label_train[train_index]

network = RBF(centers, data_train, label_train, data_test)
output = network.train()

output_valid_raw, output_valid_acc = network.valid(data_valid, label_valid)
output_test_raw, output_test = network.test()
_ = 1

threshold=-0.50 | acc = 0.86235
threshold=-0.40 | acc = 0.89879
threshold=-0.30 | acc = 0.92308
threshold=-0.20 | acc = 0.93522
threshold=-0.10 | acc = 0.94332
threshold=-0.00 | acc = 0.95547
threshold=0.10 | acc = 0.95951
threshold=0.20 | acc = 0.95142
threshold=0.30 | acc = 0.94332
threshold=0.40 | acc = 0.93117
threshold=0.50 | acc = 0.90283
********************

Choose 0.10 as threshold

********************

threshold=-0.50 | acc = 0.81928
threshold=-0.40 | acc = 0.83133
threshold=-0.30 | acc = 0.86747
threshold=-0.20 | acc = 0.87952
threshold=-0.10 | acc = 0.89157
threshold=-0.00 | acc = 0.89157
threshold=0.10 | acc = 0.90361
threshold=0.20 | acc = 0.90361
threshold=0.30 | acc = 0.93976
threshold=0.40 | acc = 0.93976
threshold=0.50 | acc = 0.91566
********************

Choose 0.30 as threshold

********************

Label of testing data are:
[ 1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1. -1.  1.  1.  1. -1.  1. -1.
  1. -1.  1.]
